In [ ]:
!pip install transformers

In [ ]:
import torch
torch.__version__

'2.1.0+cu121'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/tapex-large-finetuned-wtq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/246 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

In [ ]:
data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
table = pd.DataFrame.from_dict(data)
questions = "how many movies does Leonardo Di Caprio have?"
encoding = tokenizer(table, questions, padding=True, return_tensors="pt")
print(encoding)
print(len(encoding['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
for key, value in encoding.items():
    print('{}: {}'.format(key, value))

{'input_ids': tensor([[    0,   141,   171,  4133,   473,  2084,   261,  6782,  2269,  2927,
         12834,    33,   116, 11311,  4832,  5552,  1721,   346,     9,  4133,
          3236,   112,  4832,  5378,   625,   181,  2582,  1721,  8176,  3236,
           132,  4832,  2084,   261,  6782,  2269,  2927, 12834,  1721,  4268,
          3236,   155,  4832,  5473, 26875, 42771,  6071,  1721,  5913,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
50
['<s>', 'Ġhow', 'Ġmany', 'Ġmovies', 'Ġdoes', 'Ġle', 'on', 'ardo', 'Ġdi', 'Ġcap', 'rio', 'Ġhave', '?', 'Ġcol', 'Ġ:', 'Ġactors', 'Ġ|', 'Ġnumber', 'Ġof', 'Ġmovies', 'Ġrow', 'Ġ1', 'Ġ:', 'Ġbr', 'ad', 'Ġp', 'itt', 'Ġ|', 'Ġ87', 'Ġrow', 'Ġ2', 'Ġ:', 'Ġle', 'on', 'ardo', 'Ġdi', 'Ġcap', 'rio', 'Ġ|', 'Ġ53', 'Ġrow', 'Ġ3', 'Ġ:', 'Ġge', 'orge', 'Ġclo', 'oney', 'Ġ|', 'Ġ69', '</s>']
input_ids: tensor(

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('train_tables.pkl', 'rb') as f:
  x = pickle.load(f)
with open('dev_tables_new.pkl', 'rb') as f:
  y = pickle.load(f)
with open('question_answer_pairs_new.pkl', 'rb') as f:
  z = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'train_tables.pkl'

In [ ]:
#qlist = []
#Table only
c = 0
c2 = 0
s = 0
#f1 = 0
for i in range(50):
  qlist = []
  gold_list = []
  for j in range(len(z[i])):
    qlist.append(z[i][j]['ques'])
    gold_list.append(z[i][j]['ans'])

  encoding = tokenizer(y[i], qlist, padding=True, return_tensors="pt")
  print(len(encoding['input_ids'][0]))
  s += len(encoding['input_ids'][0])
  if(len(encoding['input_ids'][0])>512):
    c2+=1
  outputs = model.generate(**encoding)
  x = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(x)
  print(gold_list)
  print()
  '''
  common = Counter(x) & Counter(gold_list)
  num_same = sum(common.values())
  precision = 1.0 * num_same / len(x)
  recall = 1.0 * num_same / len(gold_list)
  if(num_same != 0):
    f1 += (2 * precision * recall) / (precision + recall)
  '''
  for j in range(len(z[i])):
    if(x[j].lower() == " " + gold_list[j].lower() or x[j].lower() == " " + gold_list[j].lower() + "0" or x[j].lower() == " [bold] " + gold_list[j].lower()):
      c+=1
  print(c)
  print("----------")
  print(i)
  print("***************")
print(c)
print("Exact match:")
print(c/199)
print(c2)
print(s)
#print(f1)

In [ ]:
dfx = pd.read_json('test.json')
dfx = dfx.T
dfx

In [ ]:
val = []
for i in range(100):
  val.append(dfx['table_caption'][i])
val2 = []
for i in range(100):
  val2.append(dfx['text'][i])


In [ ]:
for i in range(len(val)):
  val[i] = val[i].replace('\xa0','')

for i in range(len(val2)):
  val2[i] = val2[i].replace(' [CONTINUE] ','')

In [ ]:
import copy
a = copy.copy(z)

In [ ]:
for i in range(len(a)):
  for j in range(len(a[i])):
    a[i][j]['ques'] = a[i][j]['ques'] + " " + str(val[i])

In [ ]:
#qlist = []
#Table + caption
c = 0
c2 = 0
len_to_truncate = []
for i in range(50):
  qlist = []
  gold_list = []
  for j in range(len(a[i])):
    qlist.append(a[i][j]['ques'])
    gold_list.append(a[i][j]['ans'])

  encoding = tokenizer(y[i], qlist, padding=True, truncation=True, return_tensors="pt")
  print(len(encoding['input_ids'][0]))
  print(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  len_to_truncate.append(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  if(len(encoding['input_ids'][0])>1024):
    c2+=1
  outputs = model.generate(**encoding)

  x = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(x)
  print(gold_list)

  for j in range(len(a[i])):
    if(x[j].lower() == " " + gold_list[j].lower() or x[j].lower() == " " + gold_list[j].lower() + "0" or x[j].lower() == " [bold] " + gold_list[j].lower()):
      c+=1
  print(c)
  print("----------")
  print(i)
  print("***************")
print(c)
print("Exact match:")
print(c/199)
print(c2)

In [ ]:
import copy
b = copy.copy(a)

In [ ]:
for i in range(len(b)):
  for j in range(len(b[i])):
    b[i][j]['ques'] = b[i][j]['ques'] + " " + str(val2[i])

In [ ]:
print(a[1][2]['ques'])
print(b[1][2]['ques'])

In [ ]:
#qlist = []
#Table + caption + mention
c = 0
c2 = 0
len_to_truncate = []
for i in range(50):
  qlist = []
  gold_list = []
  for j in range(len(b[i])):
    qlist.append(b[i][j]['ques'])
    gold_list.append(b[i][j]['ans'])

  encoding = tokenizer(y[i], qlist, padding=True, truncation=True, return_tensors="pt")
  print(len(encoding['input_ids'][0]))
  print(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  len_to_truncate.append(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  if(len(encoding['input_ids'][0])>=1024):
    print(i)
    c2+=1
  outputs = model.generate(**encoding)

  x = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(x)
  print(gold_list)

  for j in range(len(b[i])):
    if(x[j].lower() == " " + gold_list[j].lower() or x[j].lower() == " " + gold_list[j].lower() + "0" or x[j].lower() == " [bold] " + gold_list[j].lower()):
      c+=1
  print(c)
  print("----------")
  print(i)
  print("***************")
print(c)
print("Exact match:")
print(c/199)
print(c2)

In [ ]:
encoding_for_text = tokenizer(y[7], b[9][2]['ques'][:946], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[9][1]['ques'][:940], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[9][0]['ques'][:941], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[9][3]['ques'][:942], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][3]['ques'][:920], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][2]['ques'][:920], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][1]['ques'][:929], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][0]['ques'][:911], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
#Truncation
b[7][0]['ques'] = b[7][0]['ques'][:998]
b[7][1]['ques'] = b[7][1]['ques'][:1002]
b[7][2]['ques'] = b[7][2]['ques'][:1005]
b[7][3]['ques'] = b[7][3]['ques'][:1002]

b[9][0]['ques'] = b[9][0]['ques'][:941]
b[9][1]['ques'] = b[9][1]['ques'][:940]
b[9][2]['ques'] = b[9][2]['ques'][:946]
b[9][3]['ques'] = b[9][3]['ques'][:942]

b[20][0]['ques'] = b[20][0]['ques'][:911]
b[20][1]['ques'] = b[20][1]['ques'][:929]
b[20][2]['ques'] = b[20][2]['ques'][:920]
b[20][3]['ques'] = b[20][3]['ques'][:920]



In [ ]:
print(b[20][0]['ques'][:911])